In [1]:
# Step 0: Mount Google Drive to save models and access project files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/StockPricePrediction


/content/drive/MyDrive/StockPricePrediction


Step 1: Install Required Libraries

In [4]:
# Step 1: Install all required packages
!pip install yfinance pandas numpy scikit-learn tensorflow textblob pytrends requests python-dotenv plotly joblib


Step 2: Import Modules

In [7]:
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 12.1 MB/s eta 0:00:00


In [8]:
# Step 2: Import required Python modules
import os
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Import your backend modules
from backend.stock_data import fetch_historical_prices
from backend.sentiment import fetch_news_sentiment, fetch_twitter_sentiment, fetch_google_trends



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
from backend.features import create_features

FEATURE_COLUMNS = ['MA10', 'MA50', 'Pct_change', 'Volatility', 'News_sentiment', 'Twitter_sentiment', 'Trend_score']


Step 3: Define Configs

In [10]:
# Step 3: Configuration for training
SECTOR_STOCKS = {
    "Technology": ["AAPL", "AMZN", "MSFT", "GOOGL", "FB"],
    "Pharma": ["PFE", "JNJ", "MRK", "ABBV"],
    "Banking": ["HDFCBANK.NS", "ICICIBANK.NS", "AXISBANK.NS", "KOTAKBANK.NS", "SBIN.NS"]
}


MODEL_DIR = "models"
WINDOW = 50       # sequence length for LSTM
EPOCHS = 50
BATCH_SIZE = 16
PERIOD = '2y'     # historical period to fetch

# Create folder to save models if it doesn't exist
os.makedirs(MODEL_DIR, exist_ok=True)


Step 4: Build LSTM Model Function

In [11]:
# Step 4: Function to build LSTM model with multiple layers and dropout
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(60, activation='relu', return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(80, activation='relu', return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(50, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [12]:
# Step 5: Function to train LSTM for a single stock
def train_lstm_for_ticker(ticker):
    print(f"\n=== TRAINING {ticker} ===")

    # Fetch historical prices and sentiment features
    hist = fetch_historical_prices(ticker, period=PERIOD)
    news = fetch_news_sentiment(ticker)
    twitter = fetch_twitter_sentiment(ticker)
    trends = fetch_google_trends(ticker)

    # Create feature dataframe
    df = create_features(hist, news, twitter, trends)
    if df.empty or len(df) < WINDOW + 10:
        print(f"Not enough data for {ticker}")
        return

    # Prepare feature matrix and scale
    data = df[FEATURE_COLUMNS].copy()
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)

    # Train/test split
    split_idx = int(len(data_scaled)*0.8)
    train_scaled = data_scaled[:split_idx]
    test_scaled = data_scaled[split_idx-WINDOW:]  # keep WINDOW for sequence

    # Function to create sequences for LSTM
    def create_sequences(dataset):
        X, y = [], []
        for i in range(WINDOW, len(dataset)-1):
            X.append(dataset[i-WINDOW:i])
            y.append(dataset[i+1, 0])  # next day's Close
        return np.array(X), np.array(y)

    X_train, y_train = create_sequences(train_scaled)
    X_test, y_test = create_sequences(test_scaled)

    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

    # Build model
    model = build_lstm_model((X_train.shape[1], X_train.shape[2]))

    # Early stopping to prevent overfitting
    es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=1
    )

    # Evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{ticker} Evaluation -> MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

    # Save model and scaler
    model_path = os.path.join(MODEL_DIR, f"{ticker}_lstm.keras")
    scaler_path = os.path.join(MODEL_DIR, f"{ticker}_scaler.save")
    model.save(model_path)
    joblib.dump(scaler, scaler_path)
    print(f"Saved model to {model_path} and scaler to {scaler_path}")


In [13]:
# Step 6: Train LSTM for all selected stocks in each sector
for sector, tickers in SECTOR_STOCKS.items():
    print(f"\n=== SECTOR: {sector} ===")
    for ticker in tickers:
        train_lstm_for_ticker(ticker)

print("✅ All sector models trained and saved.")


=== SECTOR: Technology ===

=== TRAINING AAPL ===
Twitter sentiment error: 'str' object has no attribute 'get'
Google Trends error: The request failed: Google returned a response with code 429
X_train: (350, 50, 7), y_train: (350,)
X_test: (100, 50, 7), y_test: (100,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 473ms/step - loss: 0.4623 - mae: 0.6355 - val_loss: 0.2809 - val_mae: 0.5266
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.1879 - mae: 0.4021 - val_loss: 0.0246 - val_mae: 0.1485
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0259 - mae: 0.1295 - val_loss: 0.0217 - val_mae: 0.1378
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0282 - mae: 0.1325 - val_loss: 0.0163 - val_mae: 0.1168
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0247 - mae: 0.1244 - val_loss: 0.0099 - val_mae: 0.0853
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0229 - mae: 0.1184 - val_loss: 0.0061 - val_mae: 0.0589
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0222 - mae: 0.1213 - val_loss: 0.0081 - val_mae: 0.0738
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0191 - mae: 0.1125 - val_loss: 0.0039 - val_mae: 0.0452
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Twitter sentiment error: 'str' object has no attribute 'get'
X_train: (350, 50, 7), y_train: (350,)
X_test: (100, 50, 7), y_test: (100,)
Epoch 1/50


...

22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 342ms/step - loss: 0.3856 - mae: 0.5518 - val_loss: 0.0022 - val_mae: 0.0399
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0294 - mae: 0.1356 - val_loss: 0.0218 - val_mae: 0.1421
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0229 - mae: 0.1243 - val_loss: 0.0035 - val_mae: 0.0478
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0180 - mae: 0.1046 - val_loss: 0.0031 - val_mae: 0.0456
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0167 - mae: 0.1051 - val_loss: 0.0017 - val_mae: 0.0384
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0202 - mae: 0.1152 - val_loss: 0.0115 - val_mae: 0.0996
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0150 - mae: 0.0977 - val_loss: 0.0047 - val_mae: 0.0561
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0141 - mae: 0.0994 - val_loss: 0.0060 - val_mae: 0.0665
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0153 - mae: 

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 333ms/step - loss: 0.4638 - mae: 0.6222 - val_loss: 0.0015 - val_mae: 0.0336
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0408 - mae: 0.1606 - val_loss: 0.0170 - val_mae: 0.1262
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0305 - mae: 0.1388 - val_loss: 0.0045 - val_mae: 0.0589
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0211 - mae: 0.1167 - val_loss: 0.0126 - val_mae: 0.1075
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0164 - mae: 0.1030 - val_loss: 0.0278 - val_mae: 0.1637
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0179 - mae: 0.1071 - val_loss: 0.0098 - val_mae: 0.0934
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 401ms/step
KOTAKBANK.NS Evaluation -> MSE: 0.0015, RMSE: 0.0389, MAE: 0.0336
Saved model to models/KOTAKBANK.NS_lstm.keras and scaler to models/KOTAKBANK.NS_scaler.save

=== TRAINING SBIN.NS ===
Twitter sentiment error: 'str' object has no attribute 'get'
Google Trends error: Th

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 334ms/step - loss: 0.5647 - mae: 0.7152 - val_loss: 0.2206 - val_mae: 0.4694
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1019 - mae: 0.2784 - val_loss: 0.0529 - val_mae: 0.2294
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0459 - mae: 0.1778 - val_loss: 0.0201 - val_mae: 0.1407
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0274 - mae: 0.1359 - val_loss: 0.0121 - val_mae: 0.1087
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0238 - mae: 0.1234 - val_loss: 0.0043 - val_mae: 0.0630
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0235 - mae: 0.1223 - val_loss: 0.0113 - val_mae: 0.1050
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0191 - mae: 0.1087 - val_loss: 0.0046 - val_mae: 0.0656
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0219 - mae: 0.1201 - val_loss: 0.0095 - val_mae: 0.0959
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0183 - mae: 